In [1]:
# import libraries 
import pandas as pd 
import numpy as np 
import os 

In [2]:
# import functions from sandbox src code files 
from sandbox_main_src_funs.merge_mp_yolo_dfs import (merge_mp_pose_world, clean_mp_yolo_missing_data, add_orientation_and_turn_direction, save_merge_mp_yolo_df)

from sandbox_main_src_funs.frames_to_time import (get_frames_per_second, add_time_column, save_df_w_time)

from sandbox_main_src_funs.landmark_visibility import (mp_vis_all_labels_boxplot, mp_vis_lineplot, mp_save_vis_stats_by_label, yolo_vis_lineplot)

from sandbox_main_src_funs.segment_video_walk_turn import filter_landmark_single_axis, segment_video_walks_turn

from sandbox_main_src_funs.filter_interpolate_funs import (interpolate_landmark_single_axis, 
filter_landmark_single_axis) 



In [3]:
# input - eventually will need to edit this to incorporate into main hva pipeline 

# path to video 
vid_in_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\hva_code\HomeVideoAnalysis\tests\fixtures\all_videos\RB_HC_practice videos\RB_HC_gait_vertical_right.MOV' # vid_in_path set during process_dir() of run.py

# load mp, mp_world, and yolo .csv files for one video (from main branch, output of home video analysis run.py 8/29/2024)
mp_pose_filepath = r'..\temp\main_branch_outputs\000_run\RB_HC_gait_vertical_right_mediapipe.csv'
mp_world_filepath = r'..\temp\main_branch_outputs\000_run\RB_HC_gait_vertical_right_mediapipe_world.csv'
yolo_filepath = r'..\temp\main_branch_outputs\000_run\RB_HC_gait_vertical_right_yolo.csv'

mp_pose_basename = os.path.splitext(os.path.basename(mp_pose_filepath))[0]
mp_all_filepath = mp_pose_basename + '_all.csv'

# output folder 
output_parent_folder = r'..\temp\test_sandbox_pipeline_outputs'

# read csv 
mp_pose_df = pd.read_csv(mp_pose_filepath)
mp_world_df = pd.read_csv(mp_world_filepath)
yolo_df = pd.read_csv(yolo_filepath)

In [4]:
# test input for new run.py raw pose data outpr
raw_pose_data_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\data_bw_participants\test_folder_structure'

In [5]:
# step 001 - merge mp df, add tasks info and negative Y
[mp_all_df, yolo_df] = merge_mp_pose_world(mp_pose_df, mp_world_df, yolo_df)
[mp_all_df, yolo_df] = clean_mp_yolo_missing_data(mp_all_df, yolo_df)
[mp_all_df, yolo_df] = add_orientation_and_turn_direction(vid_in_path, mp_all_df, yolo_df)
save_merge_mp_yolo_df(mp_all_df, yolo_df, vid_in_path, output_parent_folder)

# outputs 
# mp_all_df and yolo_df pandas data frames updated and .csv file saved    

In [6]:
# step 002 - get frames per second and add time column 
fps = get_frames_per_second(vid_in_path)
[mp_all_df, yolo_df] = add_time_column(mp_all_df, yolo_df, fps)
save_df_w_time(mp_all_df, yolo_df, vid_in_path, output_parent_folder)
# outputs 
# fps = video frames per second 
# [mp_all_df, yolo_df]: panda data frames added seconds and .csv file saved 

In [7]:
# step 003 - plot and save landmark visibility scores 

# mediapipe 
# boxplot 
mp_boxplot = mp_vis_all_labels_boxplot(mp_all_df, vid_in_path, output_parent_folder)

# lineplot 
mp_lineplot = mp_vis_lineplot(mp_all_df, vid_in_path, output_parent_folder)

# calculate and save vis score per label 
vis_stats_df = mp_save_vis_stats_by_label(mp_all_df, vid_in_path, output_parent_folder)

# yolo 
yolo_vis_lineplot(yolo_df, vid_in_path, output_parent_folder)

# outputs: save boxplot and lineplot .png and vis_stats_df with mean, standard deviation, and median visibility for each marker

label is na: skipped


C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\hva_code\HomeVideoAnalysis\megan_sandbox\sandbox_main_src_funs\landmark_visibility.py:150: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vis_stats_df = pd.concat([vis_stats_df, current_vis_stats_row])


skip: no_labels_tracked, included in each label


In [11]:
# Segment turn, toward, or away from camera 

max_gap = 0.12 # max gap to interpolate over 
cutoff = 0.4
order = 1

# MEDIAPIPE HIP DATA 
# Interpolate 
r_hip_mp_z_interp_df = interpolate_landmark_single_axis(mp_all_df, # mediapipe data frame 
                                                       'right_hip', # marker to interpolate 
                                                       'Z_pose', # axis to interpolate 
                                                       max_gap, # seconds, maximum gap to interpolate over 
                                                       fps,
                                                       vid_in_path, 
                                                       output_parent_folder,
                                                      mediapipe_or_yolo = 'mediapipe')

l_hip_mp_z_interp_df = interpolate_landmark_single_axis(mp_all_df, # mediapipe data frame 
                                                       'left_hip', # marker to interpolate 
                                                       'Z_pose', # axis to interpolate 
                                                       max_gap, # seconds, maximum gap to interpolate over 
                                                       fps,
                                                       vid_in_path, 
                                                       output_parent_folder, 
                                                      mediapipe_or_yolo = 'mediapipe')

# filter 
r_hip_mp_z_interp_filt = filter_landmark_single_axis(r_hip_mp_z_interp_df.iloc[:, 2],  # one series, position data from one axis of one landmark
                                                  fps, # video HZ
                                                  cutoff, # filter cutoff 
                                                  order, # butterworth filter order
                                                  vid_in_path,
                                                  output_parent_folder
                                                  )

l_hip_mp_z_interp_filt = filter_landmark_single_axis(l_hip_mp_z_interp_df.iloc[:, 2],  # one series, position data from one axis of one landmark
                                                  fps, # video HZ
                                                  cutoff, # filter cutoff 
                                                  order, # butterworth filter order
                                                  vid_in_path,
                                                  output_parent_folder
                                                  )

# YOLO HIP DATA 
# interpolate  
r_hip_yolo_z_interp_df = interpolate_landmark_single_axis(yolo_df, # mediapipe data frame 
                                                       'right_hip', # marker to interpolate 
                                                       'X_yolo', # axis to interpolate 
                                                       max_gap, # seconds, maximum gap to interpolate over 
                                                       fps,
                                                       vid_in_path, 
                                                       output_parent_folder,
                                                      mediapipe_or_yolo = 'yolo')

l_hip_yolo_z_interp_df = interpolate_landmark_single_axis(yolo_df, # mediapipe data frame 
                                                       'left_hip', # marker to interpolate 
                                                       'X_yolo', # axis to interpolate 
                                                       max_gap, # seconds, maximum gap to interpolate over 
                                                       fps,
                                                       vid_in_path, 
                                                       output_parent_folder,
                                                      mediapipe_or_yolo = 'yolo')

# filter 


# segment code 

0
0


In [9]:
# this code works - testing alternative above  

# step 004 - segment when person is walking toward or away from camera or turning 
[turn_df, walks_df] = segment_video_walks_turn(mp_all_df, yolo_df, fps, vid_in_path, output_parent_folder,
                                               cutoff = 0.4, order = 1,
                                               find_peaks_distance = 200, # min distance between hip z distance peaks  
                                               find_peaks_prominence = 0.2, # hip Z peaks need to be greater than this value to count as a turn
                                               flattening_point_atol = 0.0025, # Distance of hip z diff away from zero to be identified as "flattening point"
                                               dist_turn_mid_to_flattening = 20) # flattening point has to be at least this many frames from turn midpoint

# outputs 
# turn_df: pandas data frame with frames for each turn start, stop, and midpoint; also saved as .csv 
# walks_df: pandas data frame with frames for each walk start, stop, and midpoint and if walk was toward or away from camera; also saved as .csv 
# 2 plots saved as .png with turn/walk start stop and hip and shoulder distances 

KeyError: 'X'